In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from backend.data_processing_service import DataProcessingService

# Initialize DataProcessingService
data_processor = DataProcessingService(seq_length=10, pred_window=1, batch_size=10)
data = np.loadtxt('data/normalized_apple_prices.csv')
# data = data_processor.get_sample_data(length=50)

# Normalize the data
data_normalized, scaler = data_processor.normalize_data(data)

# Create sequences
x_data, y_data = data_processor.create_sequences(data_normalized)

# Split data into training and test sets
x_train, x_test, y_train, y_test = data_processor.split_data(x_data, y_data, train_ratio=0.8)

# Convert data to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [2]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

torch.Size([102, 10, 1]) torch.Size([102, 1, 1]) torch.Size([26, 10, 1]) torch.Size([26, 1, 1])


In [3]:
from backend.model_service import ModelService

In [4]:
import torch
import torch.nn as nn

class TimeSeriesGenerator(ModelService):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super(TimeSeriesGenerator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            # nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)
    
    @staticmethod
    def reshape_input(s):
        return s.permute(0, 2, 1)

class TimeSeriesDiscriminator(ModelService):
    def __init__(self, input_dim, hidden_dim):
        super(TimeSeriesDiscriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)
    
    @staticmethod
    def reshape_input(s):
        return s.permute(0, 2, 1)



In [7]:
class GAN(ModelService):
    def __init__(self, generator, discriminator, gen_input_dim, data_dim, lr=0.0002, betas=(0.5, 0.999)):
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.gen_input_dim = gen_input_dim
        self.data_dim = data_dim
        
        self.criterion = nn.MSELoss()
        self.optimizer_g = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=betas)
        self.optimizer_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=betas)

    def train(self, x_train, y_train, epochs):
        for epoch in range(epochs):
            for i in range(len(x_train)):
                real_data = x_train[i]
                real_labels = y_train[i].unsqueeze(1)

                noise = real_data.t()
                # Train Discriminator
                self.optimizer_d.zero_grad()
                
                # Real data
                real_output = self.discriminator(real_data.t())         # Extract reshaping too the Discrimiator class
                real_loss = self.criterion(real_output, real_labels.squeeze(0))
                
                # Fake data
                # noise = torch.randn(1, self.gen_input_dim)
                fake_data = self.generator(noise)
                fake_labels = torch.zeros(1, 1)  # Fake labels are 0
                fake_output = self.discriminator(fake_data)
                fake_loss = self.criterion(fake_output, fake_labels)
                
                # Total discriminator loss
                d_loss = real_loss + fake_loss
                d_loss.backward()
                self.optimizer_d.step()
                
                # Train Generator
                self.optimizer_g.zero_grad()
                
                # noise = torch.randn(1, self.gen_input_dim)
                fake_data = self.generator(noise)
                fake_output = self.discriminator(fake_data)
                g_loss = self.criterion(fake_output, real_labels)
                
                g_loss.backward()
                self.optimizer_g.step()
            
            print(f'Epoch [{epoch+1}/{epochs}]  Loss D: {d_loss.item()}, Loss G: {g_loss.item()}')


In [8]:
# Usage example:
# Define model parameters
gen_input_dim = 10
# data_dim = 30  # Length of the time series
input_size = data_processor.seq_length
hidden_dim = 64
# output_size = data_processor.pred_window

# Instantiate models
# generator = TimeSeriesGenerator(gen_input_dim, data_dim, hidden_dim)
generator = TimeSeriesGenerator(input_dim=gen_input_dim, output_dim=input_size, hidden_dim=hidden_dim)
# discriminator = TimeSeriesDiscriminator(data_dim, hidden_dim)
discriminator = TimeSeriesDiscriminator(input_dim=input_size, hidden_dim=hidden_dim)
gan = GAN(generator, discriminator, gen_input_dim, input_size)

# Train the GAN model
epochs = 10
gan.train(x_train, y_train, epochs)

d:\projects\times_fw\.venv\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1, 1, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10]  Loss D: 0.4205183982849121, Loss G: 0.22290702164173126
Epoch [2/10]  Loss D: 0.15232141315937042, Loss G: 0.03129754215478897
Epoch [3/10]  Loss D: 0.0774698406457901, Loss G: 0.012773692607879639
Epoch [4/10]  Loss D: 0.04704451188445091, Loss G: 0.009640452452003956
Epoch [5/10]  Loss D: 0.031687673181295395, Loss G: 0.008547845296561718
Epoch [6/10]  Loss D: 0.02274084836244583, Loss G: 0.008011503145098686
Epoch [7/10]  Loss D: 0.017758162692189217, Loss G: 0.007759559899568558
Epoch [8/10]  Loss D: 0.014956362545490265, Loss G: 0.0076571376994252205
Epoch [9/10]  Loss D: 0.013363776728510857, Loss G: 0.007603437174111605
Epoch [10/10]  Loss D: 0.012574374675750732, Loss G: 0.0075658755376935005


In [13]:
x_test[-3], y_test[-3]

(tensor([[-0.0986],
         [ 0.0676],
         [-0.0713],
         [-0.0696],
         [-0.0396],
         [-0.0436],
         [ 0.0216],
         [ 0.0265],
         [-0.0417],
         [-0.0789]]),
 tensor([[-0.0580]]))

In [14]:
# 2. Pass test data through discriminator (optional)
test_predictions = gan.discriminator(x_test[-3].t())

# 3. Evaluate generated data using discriminator (optional)
generated_predictions = gan.discriminator(x_test[-3].t())
test_predictions, generated_predictions

(tensor([[0.0586]], grad_fn=<SigmoidBackward0>),
 tensor([[0.0586]], grad_fn=<SigmoidBackward0>))

In [21]:
a = 1
x_test[-a], y_test[-a]

(tensor([[-0.0713],
         [-0.0696],
         [-0.0396],
         [-0.0436],
         [ 0.0216],
         [ 0.0265],
         [-0.0417],
         [-0.0789],
         [-0.0580],
         [ 0.2306]]),
 tensor([[0.3360]]))

In [22]:
# 2. Pass test data through discriminator (optional)
test_predictions = gan.discriminator(x_test[-a].t())

# 3. Evaluate generated data using discriminator (optional)
generated_predictions = gan.discriminator(x_test[-a].t())
test_predictions, generated_predictions

(tensor([[0.0744]], grad_fn=<SigmoidBackward0>),
 tensor([[0.0744]], grad_fn=<SigmoidBackward0>))